In [20]:
import pandas as pd
from pulp import LpProblem, LpVariable, LpBinary, LpMinimize, lpSum
from sklearn.preprocessing import MinMaxScaler
from geopy.distance import geodesic
import folium
import random

In [23]:
bus_df = pd.read_csv("../노선_요약.csv")
subway_df = pd.read_csv("../지하철_요약.csv")
cafe_df = pd.read_csv("../일반음식점(카페)현황.csv", encoding="CP949")
park_df = pd.read_csv("../도시공원정보현황(제공표준).csv", encoding="CP949")
trash_bin_df = pd.read_csv("../경기도 성남시_쓰레기통_설치현황_20250325.csv")

# 정제 및 컬럼 통일
cafe_df = cafe_df[(cafe_df['시군명'] == '성남시') & (cafe_df['영업상태명'] == '영업')]
cafe_df = cafe_df.rename(columns={'WGS84위도': '위도', 'WGS84경도': '경도', '사업장명': '이름'})

park_df = park_df[park_df['소재지지번주소'].str.startswith('경기도 성남시')].dropna(subset=['위도', '경도'])
park_df = park_df.rename(columns={'공원명': '이름'})

subway_df = subway_df.rename(columns={'역사명': '이름'})
bus_df = bus_df.dropna().rename(columns={'정류장명': '이름'})

# 가중치 계산
scaler = MinMaxScaler()
cafe_df['가중치'] = scaler.fit_transform(cafe_df[['총시설규모(㎡)']])
park_df['가중치'] = scaler.fit_transform(park_df[['공원면적(㎡)']])
bus_df['혼잡도'] = bus_df['환승시간(분)'] * bus_df['노선개수']
bus_df['가중치'] = scaler.fit_transform(bus_df[['혼잡도']])
subway_df['일평균승하차인원'] = subway_df['승차총승객수'] + subway_df['하차총승객수']
subway_df['가중치'] = scaler.fit_transform(subway_df[['일평균승하차인원']])

# 수요지 통합
demand_df = pd.concat([
    cafe_df[['위도', '경도', '이름', '가중치']],
    park_df[['위도', '경도', '이름', '가중치']],
    subway_df[['위도', '경도', '이름', '가중치']],
    bus_df[['위도', '경도', '이름', '가중치']]
], ignore_index=True)

# 기존 쓰레기통 제외한 후보지 구성
existing_bins = set(zip(trash_bin_df['위도'].round(6), trash_bin_df['경도'].round(6)))
all_candidates_df = pd.concat([
    cafe_df[['위도', '경도', '이름']],
    park_df[['위도', '경도', '이름']],
    subway_df[['위도', '경도', '이름']],
    bus_df[['위도', '경도', '이름']]
], ignore_index=True)
all_candidates_df['좌표'] = list(zip(all_candidates_df['위도'].round(6), all_candidates_df['경도'].round(6)))
new_candidates_df = all_candidates_df[~all_candidates_df['좌표'].isin(existing_bins)]

# 샘플링
demand_sample = random.sample(list(zip(demand_df['위도'], demand_df['경도'], demand_df['가중치'], demand_df['이름'])), 600)
candidate_sample = random.sample(list(zip(new_candidates_df['위도'], new_candidates_df['경도'], new_candidates_df['이름'])), 600)

In [24]:
distance_matrix = [
    [geodesic(dp[:2], cb[:2]).meters for cb in candidate_sample]
    for dp in demand_sample
]

num_demands = len(demand_sample)
num_candidates = len(candidate_sample)
p = 600  # 개수


In [25]:
x = [LpVariable(f"x_{j}", cat=LpBinary) for j in range(num_candidates)]
y = [[LpVariable(f"y_{i}_{j}", cat=LpBinary) for j in range(num_candidates)] for i in range(num_demands)]

model = LpProblem("P_Median", LpMinimize)
model += lpSum(distance_matrix[i][j] * y[i][j] for i in range(num_demands) for j in range(num_candidates))


In [26]:
for i in range(num_demands):
    model += lpSum(y[i][j] for j in range(num_candidates)) == 1

for i in range(num_demands):
    for j in range(num_candidates):
        model += y[i][j] <= x[j]

model += lpSum(x[j] for j in range(num_candidates)) <= p


In [27]:
model.solve()

selected_locations = []
for j in range(num_candidates):
    if x[j].varValue > 0.5:
        lat, lon, name = candidate_sample[j]
        selected_locations.append((lat, lon, name))

print(f"\n설치 위치 {len(selected_locations)}개")
for loc in selected_locations[:10]:
    print(f"설치 위치: {loc[2]} ({loc[0]:.5f}, {loc[1]:.5f})")



설치 위치 600개
설치 위치: 남위례역.창곡교차로 (37.46420, 127.14138)
설치 위치: 이매역 (37.39672, 127.12803)
설치 위치: 위례중앙초등학교 (37.47135, 127.13420)
설치 위치: 파크뷰아파트.청소년수련관 (37.37252, 127.10710)
설치 위치: 상원초등학교.양지동청소년문화의집 (37.45570, 127.16192)
설치 위치: 맹산공원 (37.36942, 127.13288)
설치 위치: 성남세관.시니어산업혁신센터 (37.40815, 127.14240)
설치 위치: 운중초등학교 (37.39160, 127.06678)
설치 위치: 산성동행정복지센터 (37.45160, 127.15012)
설치 위치: 이편한세상3단지 (37.44395, 127.16385)


In [28]:
center_lat = sum([pt[0] for pt in demand_sample]) / len(demand_sample)
center_lon = sum([pt[1] for pt in demand_sample]) / len(demand_sample)
m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

for lat, lon, _, name in demand_sample:
    folium.CircleMarker(
        location=[lat, lon],
        radius=3,
        color='gray',
        fill=True,
        fill_color='orange',
        fill_opacity=0.5,
        tooltip=name
    ).add_to(m)

for lat, lon, name in selected_locations:
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color='blue', icon='trash', prefix='fa'),
        tooltip=f"설치 위치: {name}"
    ).add_to(m)

m


In [29]:
selected_locations = []
selected_indices = []
for j in range(num_candidates):
    if x[j].varValue > 0.5:
        lat, lon, name = candidate_sample[j]
        selected_locations.append((lat, lon, name))
        selected_indices.append(j)

before_distances = []
after_distances = []

for i, dp in enumerate(demand_sample):
    demand_coord = dp[:2]
    
    dist_to_existing = [geodesic(demand_coord, bin_loc).meters for bin_loc in zip(trash_bin_df['위도'], trash_bin_df['경도'])]
    before_distances.append(min(dist_to_existing))
    
    dist_to_selected = [geodesic(demand_coord, candidate_sample[j][:2]).meters for j in selected_indices]
    after_distances.append(min(dist_to_selected))

import numpy as np
print(f"평균 거리 (기존): {np.mean(before_distances):.2f}m")
print(f"평균 거리 (p-Median): {np.mean(after_distances):.2f}m")
print(f"최대 거리 (기존): {np.max(before_distances):.2f}m")
print(f"최대 거리 (p-Median): {np.max(after_distances):.2f}m")


평균 거리 (기존): 1093.71m
평균 거리 (p-Median): 92.95m
최대 거리 (기존): 6436.12m
최대 거리 (p-Median): 1638.54m
